Import Libraries

In [2]:
import pandas as pd
import math


Import csv

In [4]:
df = pd.read_csv('modelinput.csv')

Helper Function to calculate distances

In [5]:
def calculate_distance(x1, y1, x2, y2):

    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

Create New Data Frame and fill in rows

In [17]:
# Initialize variables and DataFrames
final_df = pd.DataFrame()
new_row1 = {}
new_row2 = {}
new_row3 = {}
start_time = 0
seen_plays = set()  # Use a set to track seen play_ids
begun = False

# Iterate through rows in df
for index, row in df.iterrows():
    play_id = row['play_id']
    
    
    if play_id not in seen_plays:
        if not begun:
            if row['event_code'] == 4:
                begun = True
                start_time = row['timestamp']
                starting_places = eval(row['player_positions'])
                
        elif row['event_code'] != 4000:

            if row['event_code'] == 2:

                # Calculate distances to starting positions
                ball_pos = eval(row['ball_pos'])
                ball_x, ball_y = ball_pos[0], ball_pos[1]
                
                distance_1 = calculate_distance(ball_x, ball_y, starting_places[0][7][0], starting_places[0][7][1])
                distance_2 = calculate_distance(ball_x, ball_y, starting_places[1][8][0], starting_places[1][8][1])
                distance_3 = calculate_distance(ball_x, ball_y, starting_places[2][9][0], starting_places[2][9][1])

                # Populate new_row dictionaries
                new_row1 = {'distance': distance_1, 'time': row['timestamp'] - start_time, 'catch': eval(row['catch_position']).get(7, 0)}
                new_row2 = {'distance': distance_2, 'time': row['timestamp'] - start_time, 'catch': eval(row['catch_position']).get(8, 0)}
                new_row3 = {'distance': distance_3, 'time': row['timestamp'] - start_time, 'catch': eval(row['catch_position']).get(9, 0)}

                # Convert dictionaries to DataFrames
                df_new_row_1 = pd.DataFrame(new_row1, index=[0])
                df_new_row_2 = pd.DataFrame(new_row2, index=[0])
                df_new_row_3 = pd.DataFrame(new_row3, index=[0])

                # Concatenate DataFrames to final_df
                final_df = pd.concat([final_df, df_new_row_1, df_new_row_2, df_new_row_3], ignore_index=True)

                # Reset new_row dictionaries
                new_row1 = {}
                new_row2 = {}
                new_row3 = {}

                seen_plays.add(play_id)
                begun = False
            
            elif row['event_code'] != 4000:

                # Calculate distances to starting positions

                ball_pos = eval(row['ball_pos'])

                if ball_pos == 0:
                    ball_pos = (0,0,0)
                
                ball_x, ball_y = ball_pos[0], ball_pos[1]
                
                distance_1 = calculate_distance(ball_x, ball_y, starting_places[0][7][0], starting_places[0][7][1])
                distance_2 = calculate_distance(ball_x, ball_y, starting_places[1][8][0], starting_places[1][8][1])
                distance_3 = calculate_distance(ball_x, ball_y, starting_places[2][9][0], starting_places[2][9][1])

                # Fill in new_row dictionaries
                new_row1 = {'distance': distance_1, 'time': row['timestamp'] - start_time, 'catch': 0}
                new_row2 = {'distance': distance_2, 'time': row['timestamp'] - start_time, 'catch': 0}
                new_row3 = {'distance': distance_3, 'time': row['timestamp'] - start_time, 'catch': 0}

                # Convert dictionaries to DataFrames
                df_new_row_1 = pd.DataFrame(new_row1, index=[0])
                df_new_row_2 = pd.DataFrame(new_row2, index=[0])
                df_new_row_3 = pd.DataFrame(new_row3, index=[0])

                # Concatenate DataFrames to final_df
                final_df = pd.concat([final_df, df_new_row_1, df_new_row_2, df_new_row_3], ignore_index=True)

                # Reset new_row dictionaries
                new_row1 = {}
                new_row2 = {}
                new_row3 = {}
                seen_plays.add(play_id)
                begun = False


Export as csv

In [16]:
print (final_df)

        distance  time  catch
0     133.321061  5500      0
1      23.774403  5500      1
2     163.004893  5500      0
3      59.623193  5150      1
4     162.019367  5150      0
...          ...   ...    ...
1039   53.267959  4250      1
1040  201.129496  4250      0
1041  157.487968  4600      0
1042   76.108510  4600      1
1043   93.392900  4600      0

[1044 rows x 3 columns]


In [13]:
final_df.to_csv('modelData_1.csv', index=False)